# LLMs Project with OpenAI Career Agent

####  ```This will take LinkedIn profile and summery as in put and genrate Career Agent```

In [215]:
import json
import os
import requests
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
from IPython.display import Markdown, display

In [216]:
load_dotenv()

True

In [217]:
PUSHOVER_USER=os.getenv('PUSHOVER_USER')
PUSHOVER_TOKEN=os.getenv('PUSHOVER_TOKEN')
PUSHOVER_URL=os.getenv('PUSHOVER_URL')

In [218]:
print(f'PUSHOVER_USER :{PUSHOVER_USER}')
print(f'PUSHOVER_TOKEN :{PUSHOVER_TOKEN}')
print(f'PUSHOVER_URL :{PUSHOVER_URL}')

PUSHOVER_USER :umnmtj7piacqgcvawarue673dzan74
PUSHOVER_TOKEN :a8j51783yiqqmt11ipiy7jpujj2ck1
PUSHOVER_URL :https://api.pushover.net/1/messages.json


In [219]:
def push(message):
    try:
        print(f"User Message :: {message}")
        pyload={"user":PUSHOVER_USER,"token":PUSHOVER_TOKEN,"message":message}
        requests.post(PUSHOVER_URL,pyload)
        print("User Message Pushed")

    except:
        print('Failed ')

In [220]:
push("hai")

User Message :: hai
User Message Pushed


In [221]:
def record_user_details(email,name="Name not provided", notes="not provided"):
    push(f"Recording interest in {name} with email {email} and notes {notest}")
    return {"recorded": "ok"}

In [222]:
def record_unknown_questions(question):
    push(f"Recording unknown question :: {question} asked that I  couldn't answer")
    return { "recorded": "ok"}

In [223]:
record_user_details_json= {
  "name": "record_user_details",
  "description": "Use this tool to record that a user is interested in  being in touch and provided an email address",
  "parameters": {
    "type": "object",
    "properties": {
      "email": {
        "type": "string",
        "description": "The email address of this user"
      },
      "name": {
        "type": "string",
        "description": "The user's name, if they provided it"
      },
      "notes": {
        "type": "string",
        "description": "Any additional information about the conversation that's worth recording to the given context!"
      }
    },
    "required": ["email"],
    "additionalProperties": False
  }
}

In [224]:
record_unknown_questions_json={
  "name": "record_unknown_questions",
  "description": "Always use this tool to record any question you can't answer, as you don't know the answer. Never answer the question yourself.",
  "parameters": {
    "type": "object",
    "properties": {
      "question": {
        "type": "string",
        "description": "This question that couldn't be answered"
      }
    },
    "required": [
      "question"
    ],
    "additionalProperties": False
  }
}

In [225]:
tools=[
    {"type": "function", "function": record_user_details_json},
    {"type": "function", "function": record_unknown_questions_json}
]

In [226]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in  being in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to the given context!"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_questions',
   'description': "Always use this tool to record any question you can't answer, as you don't know the answer. Never answer the question yourself.",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "This question that couldn't be

In [227]:
def handle_tool_call(tool_calls):
    results=[]
    for tool_call in tool_calls:
        tool_name=tool_call.function.name
        arguments=json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        if tool_name == 'record_user_details':
            result=record_user_details(**arguments)
        elif tool_name == 'record_unknown_questions':
            result=record_unknown_question(**arguments)
        results.append({
            "role": "tool",
            "content": json.dumps(result),
            "tool_call_id": tool_call.id
        })
    return results

In [228]:
globals()["record_unknown_questions"]("This is a really hard questions")

User Message :: Recording unknown question :: This is a really hard questions asked that I  couldn't answer
User Message Pushed


{'recorded': 'ok'}

In [229]:
def  handle_tool_call(tool_calls):
    results=[]
    for tool_call in tool_calls:
        tool_name=tool_call.function.name
        arguments=json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool=globals().get(tool_name)
        result=tool(**arguments) if tool else {}

        results.append({
            "role": "tool",
            "content": json.dumps(result),
            "tool_call_id":tool_call.id
        })
        return results

In [230]:
reader=PdfReader("me/Linkdin_bharatsingh.pdf")

In [231]:
linkedId=""
for page in reader.pages:
    text=page.extract_text()
    if text:
        linkedId+=text
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summery=f.read()

In [232]:
name="Bharat Singh"

In [233]:
system_prompt=f"You are acting as {name}. You are answering questions on {name}'s website, \
                particularly questions related to {name}'s career, background, skills and experiences. \
                Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
                You are given a summary of {name}'s background and LinkedIn profile, which you can use to answer questions. \
                Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
                If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to your career. \
                If the user is engaging in the disussion, try to steer them towards getting in touch via email, ask for their email and record it using your record_user_details tool"


system_prompt+=f"\n\n## Sumery:\n\n{summery} \n\n## LinkedIn Profile: \n\n{linkedId}\n\n"
system_prompt+=f" With this context, please chat with the user, always staying in character as {name}"

In [234]:
display(Markdown(system_prompt))

You are acting as Bharat Singh. You are answering questions on Bharat Singh's website,                 particularly questions related to Bharat Singh's career, background, skills and experiences.                 Your responsibility is to represent Bharat Singh for interactions on the website as faithfully as possible.                 You are given a summary of Bharat Singh's background and LinkedIn profile, which you can use to answer questions.                 Be professional and engaging, as if talking to a potential client or future employer who came across the website.                 If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to your career.                 If the user is engaging in the disussion, try to steer them towards getting in touch via email, ask for their email and record it using your record_user_details tool

## Sumery:

A passionate AWS Java cloud Native Developer wit 11+ Years
🔭 I’m currently working on Java and AWS Cloud

🌱 I’m currently learning Machine Learing and AI.

💬 Ask me about Cloud Natvie Development with java/Python and AWS Infra

🛠  Tech Stack
1x Java | Spring Boot | 2x AWS | Docker | Helm Charts| Python | Machine learning | Deep Learning | Tensorflow | AWS CDK 

## LinkedIn Profile: 

   
Contact
Changi Business Park 
Singapore  
+919528132119  (Mobile)
bharatmca2010@gmail.com
www.linkedin.com/in/
bharatsingh213  (LinkedIn)
github.com/everythingisdata1
(Blog)
medium.com/@bharat-singh-06
(Blog)
Top Skills
Large Language Models (LLM)
GitHub Copilot
Amazon Web Services (AWS)
Languages
English
Hindi
Certifications
Amazon Web Services Solutions
Architect - Associate
Oracle Certified Professional, Java
SE 6 Programmer
Honors-Awards
Excellence Performance awards
Q4-2012
Spot Appreciation 2013
Spot Appreciation
Emirates Appreciation Najm
DC-AI HackthonBharat Singh
AWS Certified (2x) | GitHub Copilot Certified | Java Certified |
Terraform Certified | Spring Boot | Application Modernization |
Serverless Architectures | Microservices | Kubernetes | Python |
Machine Learning
Singapore, Singapore
Summary
I am a results-driven software engineer with over 13 years of
experience in cloud technologies, application modernization, and full-
stack development. Passionate about building scalable, high-quality
solutions, I specialize in designing and deploying microservices
architectures, architecting cloud infrastructure with AWS and
Kubernetes, and developing robust applications using Java, Python,
machine learning, AI, and modern frameworks.
I thrive in collaborative environments, working closely with cross-
functional teams and clients to deliver innovative, end-to-end
solutions. With a strong focus on code quality, security, and
continuous improvement, I consistently deliver reliable, maintainable,
and efficient software.
Curiosity and a passion for innovation drive me to explore emerging
technologies and embrace challenging projects that push the
boundaries of what’s possible. Currently, I am expanding my
expertise in artificial intelligence and machine learning to stay ahead
of technological advancements.
Technical Skills
Languages: Java, Python, TypeScript
Frameworks: Spring, Spring Boot, Microservices, Hibernate, JPA
Machine Learning & AI: Keras, TensorFlow, OpenCV, CUDA, Flask
Frontend: Angular, JSF, JavaScript
Databases: Oracle, Couchbase, PL/SQL
Messaging/Streaming: Kafka, RabbitMQ, Spark
DevOps & Tools: Docker, Kubernetes, GPU/TPU, Maven, Git,
SourceTree,Terraform, Helm
Mapping Technologies: Esri Map
  Page 1 of 6   
BigData Tech:  Hive, Trino, Spark
Cloud Platforms: AWS
I excel in fast-paced, evolving environments and remain committed
to delivering future-ready software solutions that create real business
impact.
Experience
Standard Chartered
Lead Development
July 2023 - Present  (2 years 4 months)
Singapore, Singapore
- Designed and developed comprehensive stress testing models to ensure
system resilience and performance.
- Implemented scalable cloud solutions using AWS services, including EKS
and ECS.
- Developed robust applications and automation scripts utilizing Java, Python,
and Kotlin.
- Architected and set up cloud infrastructure on AWS to support enterprise
workloads.
- Streamlined deployment processes by implementing one-click deployments
with Kubernetes Helm.
- Designed and managed Kubernetes deployments, services, and jobs for
efficient application orchestration.
- Proactively managed code quality and addressed security vulnerabilities to
maintain high standards.
Hitachi Vantara
Senior Consultant 2
August 2021 - July 2023  (2 years)
Bangalore Urban, Karnataka, India
- Led application modernization initiatives and cloud deployments, focusing on
scalable and secure solutions.
- Architected enterprise applications and designed microservices using Java,
Python, and Docker.
- Ensured high code quality through best practices and rigorous reviews.
Key Projects:
  Page 2 of 6   
SurePoint (Single Sign-On):
Developed a secure SSO module to streamline authentication for the
SurePoint team.
Technologies: AWS, Spring Boot, Terraform, PostgreSQL, PingIdentity
BCQMx (BeyondCare Quality Monitor):
Contributed to an evidence-based quality control management system for
monitoring analyzer health and accuracy in the healthcare domain (Sysmex).
Technologies: AWS (ECS, ECR, VPC, S3, Lambda, Kinesis, CloudFront,
SQS), Microservices, Java 11, Python, PostgreSQL
Robosoft Technologies
Technical Lead
May 2020 - August 2021  (1 year 4 months)
Bengaluru, Karnataka, India
Flipkart on Robosoft Payroll 
Domain: E-commerce
Project: Armstrong
Developed and maintained a subscription management platform enabling
users to purchase bundled subscriptions with access to multiple applications.
Implemented features for automated renewals managed by Flipkart and
ensured bundled services were offered at discounted prices.
Technical Skills:
Programming: Java, Groovy, Python
Frameworks: Spring Boot, Dropwizard, Google Guice
Database: MySQL
Cloud & DevOps: AWS, Kubernetes, Docker, Helm Charts (CI/CD)
TATWEER MIDDLE EAST AND AFRICA L.L.C 
Full Stack Developer
June 2019 - March 2020  (10 months)
Abu Dhabi, United Arab Emirates
- Collaborated with clients to define project scope and requirements.
- Developed and optimized solutions for performance and scalability, including
machine learning systems for distracted driver detection.
- Contributed to web development and smart system initiatives.
- Integrated ArcGIS mapping and geofencing capabilities.
  Page 3 of 6   
- Conducted machine learning experiments and implemented computer vision
solutions.
Technologies: Java, Angular, GIS, Python, MongoDB, Computer Vision,
Machine Learning.
Emirates Group
4 years 6 months
Data Scientist
October 2017 - June 2019  (1 year 9 months)
Dubai, United Arab Emirates
Safeguard: 
This is an NLP-based solution for group safety to auto-classify the safety
incidents raised throughout the group worldwide. This solution helps the
department auto-prioritize the incidents and saves AED 1.4 million annually.
Technology: Python, NLP, Pandas, Numpy, Tensorflow, flask, and Azure
Cloud 
Project: Group Materialization Forecasting
The general behaviour of group bookings is volatile. A group materialisation
model will predict the expected materialisation rate on the reservations and will
help influence the overbooking levels to avoid empty seats due to last-minute
cancellations.
Technology:  Python, Scikit-learn, Pandas, Numpy, Flask.
Full Stack Developer
January 2015 - September 2017  (2 years 9 months)
Dubai, United Arab Emirates
Technologies: Java, Spring Boot, Spring Data, Angular, Python, Oracle,
Couchbase
Roles & Responsibilities:
- Participated in all phases of web application development, including
requirements gathering, design, development, testing, performance tuning,
and coordination between offshore and onsite teams.
- Led proof-of-concept (PoC) initiatives and delivered complex design
solutions.
- Authored technical design documents and managed system integrations.
  Page 4 of 6   
- Upgraded legacy projects to modern technologies, ensuring improved
performance and maintainability.
- Managed build and deployment processes for staging and QA environments.
Project: Hub Monitor
Technologies: Java, Angular, Spring Boot, Oracle, JUnit
Developed Hub Monitor, a decision support system for the Network Control
Department. The platform provides situational awareness, forecasting, and
operational dashboards, enabling effective management of daily operations
and proactive resolution of issues to enhance on-time aircraft performance.
Bureau Veritas Group
3 years 7 months
Senior Software Engineer
January 2013 - December 2014  (2 years)
Hong Kong, Hong Kong SAR
• Closely involved in the process of requirement gathering & clarifications with
the business team. Performed impact analysis and conducted various POCs to
confirm technical feasibility of requirements. 
• Independently worked on complex design issues and provided solutions
which was used across projects.  Such as common components or services,
Migration from Glassfish to JBoss for multiple applications, migrating old
technology stack of projects (from JSF to HTML5, JQuery and Spring) etc.
• Worked towards building a strong team by being extensively involved in the
process of new recruitment and mentoring team members. 
• Involved in design and development across various projects involving web
based technologies such as Java, Spring Boot, Hibernate, AngularJS, JSF,
Ajax, and Oracle.
Software Engineer
June 2011 - December 2012  (1 year 7 months)
Noida Area, India
Roles& Responsibility :
Requirement gathering, analysis, design, coding, deployment and
maintenance  projects.
Worked on some project from scratch. 
Client discussion.
Updating all the Mandatory Sheets to maintain the Process.
Implement new solution as per business needs. 
Involved in Unit testing and Integration Testing. 
  Page 5 of 6   
Application : -
1) BVLABNG :-(Bureau Veritas Lab NG)
BVCPS core business is product testing. So for automate test lab,  we have
implemented this application.  It holds product testing result information.   
2) CDM :- (Client Directory Management)
CDM is a centralized CPS Client Management. it's storing data in
Chinese(simplified ,), Korean, Vietnamese languages.
Tools& Technologies :-  Java,Servlet, JSP, JQuery, JSF2.0, Oracle, Glassfish
v2, JDBC, Maven, Netbean.
RoseIndia Technology
Software Engineer
July 2010 - May 2011  (11 months)
New Delhi, India
• Capture requirements for new solution as per business needs.
• Solve technical issues of business and also develop some module.
• Involved in Unit testing and Integration Testing.
• Responsible for Fixing bugs and maintenance of projects.
• Design, analysis and application development.
• Designed the logical and physical data module generates DDL Scripts and
wrote DML script s for Oracle 10g database.
Education
Dr. A.P.J. Abdul Kalam Technical University
Master of Computer Applications - MCA, Computer Science  · (2007 - 2010)
Kumaun University
Bachelor of Science - BS   · (2004 - 2007)
  Page 6 of 6

 With this context, please chat with the user, always staying in character as Bharat Singh

In [235]:
openai_client=OpenAI()
model="gpt-4o-mini"

In [236]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:
        # This is the call to the LLM- see that we pass in the tools JSON.
        response = openai_client.chat.completions.create(model=model,messages=messages,tools=tools)

        finish_reason = response.choices[0].finish_reason
        print(f"finish_reason :: {finish_reason}")

        # If LLM wants to call a tool,  we do that!
        if finish_reason == 'tool_calls':
            message = response.choices[0].message
            print(f"Tool message :: {message}")
            tool_calls = message.tool_calls
            results = handle_tool_call(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content


In [237]:
gr.ChatInterface(chat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


finish_reason :: tool_calls
Tool message :: ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_bDLfqzXBhWASGbCwvM9QSwcj', function=Function(arguments='{"question":"who is your favorite in your company?"}', name='record_unknown_questions'), type='function')])
Tool called: record_unknown_questions
User Message :: Recording unknown question :: who is your favorite in your company? asked that I  couldn't answer
User Message Pushed
finish_reason :: stop
